In [17]:
import csv
import datetime
import operator
import os
import re
from random import randint
import networkx as nx
import csv
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from operator import itemgetter
from shapely.wkt import loads
import copy
from os import path
import netwulf as nw
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mtick
from os import listdir

/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/ipykernel_73687/51224624.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [16]:
NOT_INFECTED=0
INFECTED=1
NUM_NODOS=1
PERCENTAGE = 0.60

enlaces_global = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_GrafoGlobal.csv"
enlaces_duero = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO DUERO.csv"
enlaces_tajo = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO TAJO.csv"
enlaces_guadiana = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GUADIANA.csv"
enlaces_jucar = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO JUCAR.csv"
enlaces_ebro = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO EBRO.csv"
enlaces_segura = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO SEGURA.csv"
enlaces_garona = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GARONA.csv"
enlaces_barbate = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO BARBATE.csv"
enlaces_ter =  "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIU TER.csv"
enlaces_palancia = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIU PALANCIA.csv"
enlaces_guadalquivir = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GUADALQUIVIR.csv"
rios = [(enlaces_guadalquivir,"RIO GUADALQUIVIR"),(enlaces_palancia,"RIO PALANCIA"),(enlaces_ter,"RIO TER"),(enlaces_barbate,"RIO BARBATE"),(enlaces_garona,"RIO GARONA"),(enlaces_segura,"RIO SEGURA"),(enlaces_guadiana,"RIO GUADIANA"),(enlaces_ebro,"RIO EBRO"),(enlaces_jucar,"RIO JUCAR"),(enlaces_tajo,"RIO TAJO")]

fosfato = "/Users/silviadelatorre/Desktop/TFG/DISTANCIAS SENSORES/3 COORDS/ATLANTICO/Fosfato.csv"
'''amonio
fosforo
fitobentos
nitrato
grado_trofico'''


'amonio\nfosforo\nfitobentos\nnitrato\ngrado_trofico'

In [14]:

def CrearGrafo(edge_list):
    # Crear un grafo no dirigido
    G = nx.DiGraph()

    # Analizar el edge list y agregar los nodos y las aristas al grafo
    with open(edge_list, 'r') as f:
        for line in f:
            nodos_y_atributos = line.strip().split(';')
            nodo1_coords = tuple(map(float, nodos_y_atributos[0][1:-1].split(', ')))
            nodo2_coords = tuple(map(float, nodos_y_atributos[1][1:-1].split(', ')))

            # Agregar la arista al grafo
            G.add_edge(nodo2_coords, nodo1_coords)

    print("Número de nodos:", G.number_of_nodes())
    print("Número de aristas:", G.number_of_edges())
    degrees = G.degree()
    nodes = G.nodes()
    n_color = np.asarray([degrees[n] for n in nodes])
    #GraficarRed(G,n_color)
    return G

In [32]:

def StatusNodes(V):
    global NodeInfo
    global df
    Nodes=V.nodes()
    NodeInfo = pd.DataFrame(Nodes)
    #print(NodeInfo)
    data = {'Node': Nodes, 'Status': [NOT_INFECTED] * len(Nodes)}
    df = pd.DataFrame(data)

    print(f'Status data frame: {df}')

In [12]:
def Infected_Nodes(V):
    global df

    StatusNodes(V)
    global df
    TotalNodes = len(V.nodes())
    TotalNodesInfect = df['Status'].sum()

    Rate = TotalNodesInfect/TotalNodes
    return Rate


In [25]:
def CodigodelRio(rio):
    print(rio)
    if(rio=='RIO GUADALQUIVIR'):
        codigo = '10094'
    elif(rio=='RIO GARONA'):
        codigo = '10034'
    elif(rio=='RIO MIÑO'):
        codigo = '10038'
    elif(rio=='RIO DUERO'):
        codigo = '1004'
    elif(rio=='RIO TAJO'):
        codigo = '1006'
    elif(rio=='RIO GUADIANA'):
        codigo = '1008'
    elif(rio=='RIO BARBATE'):
        codigo = '10098'
    elif(rio=='RI0 TER'):
        codigo = '20054'
    elif(rio=='RIO LLOBREGAT'):
        codigo = '20052'
    elif(rio=='RIO PALANCIA'):
        codigo = '20036'
    elif(rio=='RIO SEGURA'):
        codigo = '20016'
    elif(rio=='RIO EBRO'):
        codigo='2004'
    elif(rio=='RIO JUCAR'):
        codigo='2002'
    
    return codigo

In [31]:
def ObtenerPrimerNodo(G,agente,rio):
    max = 0
    coord_max_fosfato = None
    print(f'Nodo más contaminado de {str(agente)} del {rio}')
    codigo = CodigodelRio(rio)
    print(f"El codigo del {rio} es {codigo}")
    with open(agente, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            # Obtiene el ID del río, la coordenada y el valor de fosfato
            id_rio = row[0]
            coord = row[1]
            valor_contaminante = float(row[2])
            print(valor_contaminante)
            # Comprueba si el valor actual del agente es mayor que el máximo
            if str(id_rio).startswith(str(codigo)) and valor_contaminante > max:
                max = valor_contaminante
                coord_max = coord
                print(coord_max)
    print("Mayor valor de contaminación es",max)
    return coord_max

In [9]:
def DibujarVariasGraficas(Dir,pC, pR):
    
    #mi_path = os.getcwd() + "\\" + "..\\datos\\" + str(Dir)
    mi_path=Dir
    
    fig = plt.figure()
    ax = plt.subplot(111)
    plt.title('Failure propagation')
    ax.set_xlabel('n')
    ax.set_ylabel('Infection rate')
    fmt = '%.5f' # Format you want the ticks, e.g. '40%'
    xticks = mtick.FormatStrFormatter(fmt)
    ax.yaxis.set_major_formatter(xticks)

    datosHeader = ['iter', 'rate']
    for NomFich in listdir(mi_path):
        pathFile = mi_path + "\\" + NomFich
        datos = pd.read_table(pathFile, 'engine=python', delimiter=' ', header=0, encoding = "ISO-8859-1", names=datosHeader)
        ax.plot(datos['iter'], datos['rate'], marker='o', label='pI: '+str(pC) + ' pR: '+str(pR))
        ax.legend()
    fileFig="TodosGraficos.png"
    pathFileFig = mi_path + "\\" + fileFig
    fig.savefig(pathFileFig)

In [8]:
def ComprobarInfeccion (N, p):
    
    if (np.random.binomial(N,p) > 0):
        return INFECTED
    else:
        return NOT_INFECTED

def ComprobarRecuperacion (p):
    
    if (np.random.binomial(1,p) > 0):
        return NOT_INFECTED
    else:
        return INFECTED 

In [7]:
def VerificarInfeccionVecinos(i):

    global NodeInfo, V, pC
    
    Neighbours= V.neighbors(i)
    NumNodosInfect= NodeInfo.status[(NodeInfo['node'].isin (Neighbours)) & (NodeInfo['status']==INFECTED)].sum()
    NodeInfo.status[NodeInfo.node==i] = ComprobarInfeccion (NumNodosInfect, pC)

def AplicarRecuperacion(i):

    global NodeInfo,pR
	
    NodeInfo.status[(NodeInfo['node']==i)]=ComprobarRecuperacion(pR)

In [6]:
def EjecutarPropagacionInfeccionTime(PERCENTAGE):
    global Rate, TotalNodes, NodeInfo, V
    pathFile = "Results/PROPAGACIÓN/ResultadosPropagación.csv"
    print("Time " + str(Time))

    while Rate < PERCENTAGE:
        # Propagar infección
        [VerificarInfeccionVecinos(i) for i in NodeInfo.node[NodeInfo.status == NOT_INFECTED]]
        TotalNodesInfect = NodeInfo.status[NodeInfo.status == INFECTED].sum()

        # Aplicar recuperación
        [AplicarRecuperacion(i) for i in NodeInfo.node[NodeInfo.status == INFECTED]]

        Rate = TotalNodesInfect / TotalNodes

        DatosTime = pd.DataFrame([[0, 0]], columns=['time', 'rate'], dtype=int)
        DatosTime['time'] = Time
        DatosTime['rate'] = Rate

        DatosTime.to_csv(pathFile, header=None, index=None, mode='a', sep=' ', decimal='.')

        DatosTime = DatosTime.drop(labels="time", axis=1)
        DatosTime = DatosTime.drop(labels="rate", axis=1)

        Time += 1

    print("Infección alcanzó el " + str(PERCENTAGE * 100) + "% de los nodos INFECTEDs.")
    print("End time " + str(Time))

In [34]:

def EjecutarIteracionInfeccion(V,Tipo,rio):

    global pathFile, Time, NodeInfo, Rate, TotalNodes, NDir, Iter

    #print("Iteracion" + str(Iter))
    # print(Iter)

    print("Propagando infeccion")

    Nodes = V.nodes()
    TotalNodes = len(Nodes)
    print("Total nodes " + str(TotalNodes))
    FirstNode = ObtenerPrimerNodo(V,Tipo,rio)
    # FirstNode=Nodes[randint(1, len(Nodes))-1]
    print("Primer nodo INFECTED")
    print(FirstNode)
    #GraficarInfección(V,FirstNode)
    '''NodeInfo = pd.DataFrame(Nodes)
    NodeInfo.columns = ['Node']
    NodeInfo['Infection Status'] = NOT_INFECTED'''
    NodeInfo.Status[NodeInfo.Node==FirstNode] = INFECTED
    TotalNodesInfected = NodeInfo.Status[NodeInfo.Status==INFECTED].sum()

    Rate = TotalNodesInfected/TotalNodes
    
    pathFile = str(NDir)+ "\\" + rio + "_" + Tipo + "-"+str(FirstNode)+"-"+str(Iter)+"-"+str(pC) + "_"+ str(pR)
    pathFile=pathFile.replace('.txt', '')
    pathFile = pathFile + ".txt"
    print("Directorio"+str(pathFile))
    #if (path.exists(pathFile)):
    #    os.remove(pathFile)
	
    rate = Infected_Nodes(V)
    while (rate < PERCENTAGE):
        EjecutarPropagacionInfeccionTime(rate)

    print("Rate "+str("{0:.4f}".format(Rate)))
    print("End time "+str(Time))

In [4]:

def PropagarInfeccion2 (pR, pC, Tipo,V,rio):

    global NDir

    print("Creando directorio")
    NDir= "/Users/silviadelatorre/Desktop/TFG/PFG/Results/PROPAGACIÓN/"+str(pC)+"_"+str(pR)+"_NodoconMayorContaminacion"
    if (path.exists(NDir)):
        print("")
    else:
        #shutil.rmtree(NDir)
        os.mkdir(NDir)
    print("Ejecutando iteraciones")
    
    if (Tipo == 'Am'):
        EjecutarIteracionInfeccion(V,amonio,rio)
    elif (Tipo == 'Ni'):
        EjecutarIteracionInfeccion(V,nitrato,rio)
    elif (Tipo == 'Fr'):
        EjecutarIteracionInfeccion(V,fosforo,rio)
    elif (Tipo == 'Ft'):
        EjecutarIteracionInfeccion(V,fosfato,rio)
    elif (Tipo == 'Gt'):
        EjecutarIteracionInfeccion(V,grado_trofico,rio)
    else:
        EjecutarIteracionInfeccion(V,fitobentos,rio)


In [3]:

def PropagarInfeccionTodasProbabilidades(i, Tipo,V,rio):
    global pC, pR
    
    pC=i
    pR=0
    print("Probabilidad contagio " + str(pC))

    PropagarInfeccion2 (pR, pC, Tipo,V,rio)

In [2]:
def Menu2(graph,nombre_rio):
    while True:
        print("Choose an option:")
        print("1. Propagation with infection starting in node with highest nitrate contamination")
        print("2. Propagation with infection starting in node with highest phosphorus contamination")
        print("3. Propagation with infection starting in node with highest phosphate contamination")
        print("4. Propagation with infection starting in node with highest ammonium contamination")
        print("5. Propagation with infection starting in node with highest phytobenthos contamination")
        print("6. Propagation with infection starting in node with highest trophic grade contamination")
        print("7. Propagation with infection starting in node with highest overall contamination")
        print("8. Exit")

        option = input("")

        if option == "1":
            rate = Infected_Nodes(graph)
            if (rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Nt",graph,nombre_rio)
            break
        elif option == "2":
            rate = Infected_Nodes(graph)
            while (rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Fr",graph,nombre_rio)
            break
        elif option == "3":
            rate = Infected_Nodes(graph)
            while (rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Ft",graph,nombre_rio)
            break
        elif option == "4":
            rate = Infected_Nodes(graph)
            while (rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Am",graph,nombre_rio)
            break
        elif option == "5":
            rate = Infected_Nodes(graph)
            while (rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Fit",graph,nombre_rio)
            break
        elif option == "6":
            rate = Infected_Nodes(graph)
            while (rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Gt",graph,nombre_rio)
            break
        elif option == "7":
            break
        else:
            print("Invalid option. Please choose again.")

    return option

In [1]:

def Menu1():
    while True:
        print("Choose an option:")
        print("1. Simulate propagation for the global network of rivers")
        print("2. Simulate propagation for each individual non-connected subnetwork")
        print("3. Exit")

        option = input("")
        print("You chose option: ",option)

        if option == "1":
            print("Simulating propagation for the entire network, infecting one node per subnetwork")
            G = CrearGrafo(enlaces_global)
            Menu2(G,"RIOS ESPAÑA")
            return option
        
        elif option == "2":
            print("Simulating propagation for each individual subnetwork")
            
            for enlaces,nombre_rio in rios:
                print(f"Propagar infección en rio {nombre_rio}")
                G = CrearGrafo(enlaces)
                Menu2(G,nombre_rio)
            
            return option
        elif option == "3":
            print("Exiting")
            return option
        else:
            print("Invalid option. Please choose again.")



In [35]:
option1 = Menu1()

Choose an option:
1. Simulate propagation for the global network of rivers
2. Simulate propagation for each individual non-connected subnetwork
3. Exit
You chose option:  2
Simulating propagation for each individual subnetwork
Propagar infección en rio RIO GUADALQUIVIR
Número de nodos: 63815
Número de aristas: 66383
Choose an option:
1. Propagation with infection starting in node with highest nitrate contamination
2. Propagation with infection starting in node with highest phosphorus contamination
3. Propagation with infection starting in node with highest phosphate contamination
4. Propagation with infection starting in node with highest ammonium contamination
5. Propagation with infection starting in node with highest phytobenthos contamination
6. Propagation with infection starting in node with highest trophic grade contamination
7. Propagation with infection starting in node with highest overall contamination
8. Exit
Status data frame:                               Node  Status
0  

KeyError: 'Infection Status'